In [7]:
import pandas as pd

df = pd.read_csv('MBA.csv', low_memory=False)

# Check the datasets 

print(df.head())
print(df.info())
print(df.describe())

print(df.isnull().sum())
print(df[df.isnull().any(axis=1)])
print(df.duplicated().sum())
print(df.dtypes)




   BillNo                             Itemname  Quantity          Date  Price  \
0  536365   WHITE HANGING HEART T-LIGHT HOLDER         6  12/1/10 8:26   2.55   
1  536365                  WHITE METAL LANTERN         6  12/1/10 8:26   3.39   
2  536365       CREAM CUPID HEARTS COAT HANGER         8  12/1/10 8:26   2.75   
3  536365  KNITTED UNION FLAG HOT WATER BOTTLE         6  12/1/10 8:26   3.39   
4  536365       RED WOOLLY HOTTIE WHITE HEART.         6  12/1/10 8:26   3.39   

   CustomerID         Country  
0     17850.0  United Kingdom  
1     17850.0  United Kingdom  
2     17850.0  United Kingdom  
3     17850.0  United Kingdom  
4     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 

In [8]:
# removes all negative and zero values in quantities
df =df[df['Quantity'] > 0 ]

# remove 0 price
df = df[df['Price'] > 0]

#check products the are not real or invalid item
df[df['Price'] == 0]['Itemname'].value_counts()



Series([], Name: count, dtype: int64)

In [9]:
# remove the invalid item
invalid_items = ['check', 'found', 'Found', 'adjustment', 'amazon']
df = df[~df['Itemname'].isin(invalid_items)]

In [10]:
# drop invalid columns

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.dropna(subset=['Itemname'], inplace=True)

In [11]:
import os

if not os.path.exists("cleaned_MBA.csv"):
    df.to_csv("cleaned_MBA.csv", index=False)



In [ ]:
# Import the cleaned data
cleaned_df = pd.read_csv("cleaned_MBA.csv")

   BillNo                             Itemname  Quantity          Date  Price  \
0  536365   WHITE HANGING HEART T-LIGHT HOLDER         6  12/1/10 8:26   2.55   
1  536365                  WHITE METAL LANTERN         6  12/1/10 8:26   3.39   
2  536365       CREAM CUPID HEARTS COAT HANGER         8  12/1/10 8:26   2.75   
3  536365  KNITTED UNION FLAG HOT WATER BOTTLE         6  12/1/10 8:26   3.39   
4  536365       RED WOOLLY HOTTIE WHITE HEART.         6  12/1/10 8:26   3.39   

   CustomerID         Country  
0     17850.0  United Kingdom  
1     17850.0  United Kingdom  
2     17850.0  United Kingdom  
3     17850.0  United Kingdom  
4     17850.0  United Kingdom  


In [14]:
from mlxtend.frequent_patterns import apriori, association_rules

# Create the basket (transaction matrix) from cleaned_df
basket = cleaned_df.pivot_table(index='BillNo', columns='Itemname', values='Quantity', aggfunc='sum', fill_value=0)

# Filter out infrequent items to reduce memory usage
item_frequencies = basket.sum(axis=0)
frequent_items = item_frequencies[item_frequencies >= 20].index  # keep items bought at least 20 times
basket_filtered = basket[frequent_items]

# Convert quantities to 0 or 1 and to boolean for memory efficiency
basket_sets = basket_filtered.applymap(lambda x: 1 if x >= 1 else 0).astype(bool)

# Increase min_support if still running out of memory (try 0.01 or higher)
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

print("Frequent Itemsets:")
print(frequent_itemsets.head())

C:\Users\leyan\AppData\Local\Temp\ipykernel_2984\4156348744.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket_filtered.applymap(lambda x: 1 if x >= 1 else 0).astype(bool)


Frequent Itemsets:
    support                               itemsets
0  0.013051               (10 COLOUR SPACEBOY PEN)
1  0.010133      (12 MESSAGE CARDS WITH ENVELOPES)
2  0.015089        (12 PENCIL SMALL TUBE WOODLAND)
3  0.016796  (12 PENCILS SMALL TUBE RED RETROSPOT)
4  0.015915          (12 PENCILS SMALL TUBE SKULL)


In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)


Association Rules:


In [19]:
rules_filtered = rules[(rules['confidence'] >= 0.6) & (rules['lift'] >= 1.2)]

In [20]:
# Display top 10 rules
print("\nAssociation Rules (Top 10):")
print(rules_filtered[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
      .sort_values(by='lift', ascending=False).head(10))


Association Rules (Top 10):
                         antecedents                      consequents  \
528       (REGENCY SUGAR BOWL GREEN)          (REGENCY MILK JUG PINK)   
529          (REGENCY MILK JUG PINK)       (REGENCY SUGAR BOWL GREEN)   
531        (REGENCY TEA PLATE ROSES)        (REGENCY TEA PLATE GREEN)   
530        (REGENCY TEA PLATE GREEN)        (REGENCY TEA PLATE ROSES)   
504   (POPPY'S PLAYHOUSE LIVINGROOM)      (POPPY'S PLAYHOUSE BEDROOM)   
505      (POPPY'S PLAYHOUSE BEDROOM)   (POPPY'S PLAYHOUSE LIVINGROOM)   
560  (SET/6 RED SPOTTY PAPER PLATES)    (SET/6 RED SPOTTY PAPER CUPS)   
561    (SET/6 RED SPOTTY PAPER CUPS)  (SET/6 RED SPOTTY PAPER PLATES)   
506      (POPPY'S PLAYHOUSE KITCHEN)   (POPPY'S PLAYHOUSE LIVINGROOM)   
507   (POPPY'S PLAYHOUSE LIVINGROOM)      (POPPY'S PLAYHOUSE KITCHEN)   

      support  confidence       lift  
528  0.010133    0.763485  56.588297  
529  0.010133    0.751020  56.588297  
531  0.011399    0.687708  50.764565  
530  0.0113

Explain the shit here report it butngig visual para chada